# Revista Ambiente Contábil

## Bibliotecas importadas

In [1]:
import requests
from bs4 import BeautifulSoup

import pandas as pd

## Funções básicas

In [2]:
def criar_query(key_word):
    return "+".join(key_word.split())

In [3]:
def listar_links_pagina(url_search):
    page = requests.get(url_search)
    soup = BeautifulSoup(page.content, 'html.parser')
    results = soup.find_all('h3', {'class': 'title'})
    a_links = []
    for item in results:
        try:
            remover_links = 'Editorial' in item.find('a').text
            remover_links = remover_links or 'Folha de rosto' in item.find('a').text
            remover_links = remover_links or 'Dados da edição ' in item.find('a').text
            if not remover_links:
                a_links.append(item.find('a')['href'])
        except:
            pass
    return a_links

In [4]:
def listar_links(url_search_text, query, pg_num=1):
    a_links = [] 
    while True:
        url_search = url_search_text.replace('__query__', query).replace('__pg_num__', str(pg_num))
        tmp = listar_links_pagina(url_search)
        if len(tmp) != 0:
            pg_num += 1
            a_links += tmp
        else:
            break
    return a_links

In [5]:
url_search_text = 'https://periodicos.ufrn.br/ambiente/search/index?query=__query__&searchJournal=8&authors=&dateFromMonth=&dateFromDay=&dateFromYear=2012&dateToMonth=&dateToDay=&dateToYear=2021&searchPage=__pg_num__#results'

links = listar_links(url_search_text, 'contabilidade+ambiental', pg_num=1)

link = links[0]

link


'https://periodicos.ufrn.br/ambiente/article/view/20064'

In [6]:
links

['https://periodicos.ufrn.br/ambiente/article/view/20064',
 'https://periodicos.ufrn.br/ambiente/article/view/9052',
 'https://periodicos.ufrn.br/ambiente/article/view/3318',
 'https://periodicos.ufrn.br/ambiente/article/view/7423',
 'https://periodicos.ufrn.br/ambiente/article/view/4144',
 'https://periodicos.ufrn.br/ambiente/article/view/8997',
 'https://periodicos.ufrn.br/ambiente/article/view/21751',
 'https://periodicos.ufrn.br/ambiente/article/view/4960',
 'https://periodicos.ufrn.br/ambiente/article/view/17145',
 'https://periodicos.ufrn.br/ambiente/article/view/4209',
 'https://periodicos.ufrn.br/ambiente/article/view/4728',
 'https://periodicos.ufrn.br/ambiente/article/view/4149',
 'https://periodicos.ufrn.br/ambiente/article/view/10787',
 'https://periodicos.ufrn.br/ambiente/article/view/4122',
 'https://periodicos.ufrn.br/ambiente/article/view/7463',
 'https://periodicos.ufrn.br/ambiente/article/view/18923',
 'https://periodicos.ufrn.br/ambiente/article/view/2137',
 'https:/

In [7]:
page = requests.get(link)
soup = BeautifulSoup(page.content, 'html.parser')

In [8]:
def completar_lista(lista):
    if len(lista) < 6:
        lista += (6 - len(lista)) * ['']
    return lista

In [9]:
def limpar_afiliacao(afiliacao):
    afiliacao = ' '.join(afiliacao.split())
    afiliacao = afiliacao.split('-')[0]
    afiliacao = afiliacao.split('(')[0]
    return afiliacao.strip()

In [10]:
def captura_titulo(soup):
    try:
        title = soup.find('h1', {'class': 'page_title'}).text
        title = ' '.join(title.split())
    except:
        title = ''
    return {'Título': title}

In [11]:
captura_titulo(soup)

{'Título': 'Nível de conhecimento dos profissionais de contabilidade de Três Lagoas (MS) sobre contabilidade ambiental, gestão e legislação ambiental'}

In [12]:
def captura_data(soup):
    try:
        date_tag = soup.find('div', {'class': 'item published'})
        date = date_tag.find('div', {'class': 'value'}).text
        date = ' '.join(date.split())
    except:
        date = ''
    return {'Data de Publicação': date[:10]}

In [13]:
captura_data(soup)

{'Data de Publicação': '02-07-2021'}

In [14]:
def captura_resumo(soup):
    try:
        abstract_section = soup.find('section', {'class': 'item abstract'})
        abstract = abstract_section.text
        abstract = abstract.replace('Resumo:', '')
        abstract = abstract.replace('Resumo', '').strip()
        #abstract = ' '.join(abstract.split())
    except:
        abstract = ''
    return {'Resumo': abstract}

In [15]:
captura_resumo(soup)

{'Resumo': 'Objetivo: A preocupação com o meio ambiente é constante e vem, gradativamente, ganhando destaque: político, acadêmico e empresarial. O tema configura-se como relevante, porque para a maioria das empresas a preocupação com o meio ambiente ainda não se transformou em práticas efetivas de gestão ambiental e muitos profissionais da contabilidade têm formação anterior a inserção da contabilidade ambiental como um novo ramo de atuação das ciências contábeis. No entanto, a educação continuada se estabelece como um meio de atualização dos profissionais em relação as mudanças e novas tendências de atuação do profissional contábil. Assim, a pesquisa se propôs analisar o nível de conhecimento dos profissionais de contabilidade de Três Lagoas (MS) sobre contabilidade ambiental, gestão e legislação ambiental.\nMetodologia: A pesquisa classifica-se como quantitativa e adota como estratégia de investigação a pesquisa de levantamento de opiniões dos contadores atuantes nos escritórios de c

In [16]:
def captura_palavras_chave(soup):
    try:
        kw_section = soup.find('section', {'class': 'item keywords'})
        kw = kw_section.text.replace('Palavras-chave:', '')
        kw = ' '.join(kw.split())
        kw = kw.replace(',', '.').replace(';', '.')
    except:
        kw = ''
    if kw == '':
        try:
            kw_section = soup.find('section', {'class': 'item abstract'})
            p_list = kw_section.find_all('p')
            for p in p_list:
                if 'Palavras-chave' in p.text:
                    kw_full = p.text
                    kw = kw_full.replace('Palavras-chave', '').replace(':', '')
                    kw = ' '.join(kw.split())
                    kw = kw.replace(',', '.').replace(';', '.')
        except:
            pass
    return {'Palavras-chave': kw}

In [17]:
captura_palavras_chave(soup)

{'Palavras-chave': 'Contador. Contabilidade e Gestão ambiental. Educação continuada.'}

In [18]:
def captura_autores(soup):
    autores = []
    try:
        authors_ul = soup.find('ul', {'class': 'authors'})
        autores = authors_ul.find_all('span', {'class': 'name'})
        autores = [' '.join(a.text.split()) for a in autores]
    except:
        pass
    qtd_autores = len(autores)
    autores = completar_lista(autores)
    autores = [qtd_autores] + autores
    
    key_autores = [f"Autor {i}" for i in range(1, 7)]
    key_autores = ['Qtd. de Autores'] + key_autores

    return dict(zip(key_autores, autores))

In [19]:
captura_autores(soup)

{'Qtd. de Autores': 5,
 'Autor 1': 'Sirlei Tonello Tisott',
 'Autor 2': 'Sílvio Paula Ribeiro',
 'Autor 3': 'Tamires Sousa Araújo',
 'Autor 4': 'Cinthia da Costa Pontes',
 'Autor 5': 'Jhonatan Verissímo da Sílva',
 'Autor 6': ''}

In [20]:
def captura_afiliacao(soup):
    afiliacao = []
    try:
        authors_ul = soup.find('ul', {'class': 'authors'})
        afiliacao = authors_ul.find_all('span', {'class': 'affiliation'})
        afiliacao = [' '.join(a.text.split()) for a in afiliacao]
        afiliacao = [limpar_afiliacao(a) for a in afiliacao]
    except:
        pass
    afiliacao = completar_lista(afiliacao)
    
    key_affiliation = [f"Afiliação {i}" for i in range(1, 7)]

    return dict(zip(key_affiliation, afiliacao))

In [21]:
captura_afiliacao(soup)

{'Afiliação 1': 'Universidade Federal de Mato Grosso do Sul',
 'Afiliação 2': 'Universidade Federal de Mato Grosso do Sul',
 'Afiliação 3': 'Universidade Federal de Mato Grosso do Sul',
 'Afiliação 4': 'Universidade Federal de Mato Grosso do Sul',
 'Afiliação 5': 'Universidade Federal de Mato Grosso do Sul',
 'Afiliação 6': ''}

In [22]:
def realizar_consulta(key_words_list):
    periodico_nome_dict = {'Revista': 'Revista Ambiente Contábil'}
    url_search_text = 'https://periodicos.ufrn.br/ambiente/search/index?query=__query__&searchJournal=8&authors=&dateFromMonth=&dateFromDay=&dateFromYear=2012&dateToMonth=&dateToDay=&dateToYear=2021&searchPage=__pg_num__#results'
    a_links = []
    for key_word in key_words_list:
        query = criar_query(key_word)
        a_links += listar_links(url_search_text, query)
        
    resultados = []
    for link in a_links:
        link_dict = {'Url Artigo': link}
        page = requests.get(link)
        soup = BeautifulSoup(page.content, 'html.parser')
        dict_artigo = {
            **captura_data(soup),
            **captura_titulo(soup),
            **captura_resumo(soup),
            **captura_palavras_chave(soup),
            **captura_autores(soup),
            **captura_afiliacao(soup),
            **link_dict,
            **periodico_nome_dict
        }
        resultados.append(dict_artigo)
    df = pd.DataFrame(resultados)
    df['Data de Publicação'] = pd.to_datetime(df['Data de Publicação'], format='%d-%m-%Y')
    return df

## Pesquisa por artigos

In [23]:
key_words_list = ['contabilidade ambiental', 'balanço social', 'relato integrado', 'nbct 15']
df = realizar_consulta(key_words_list)
df.head()

,Data de Publicação,Título,Resumo,Palavras-chave,Qtd. de Autores,Autor 1,Autor 2,Autor 3,Autor 4,Autor 5,Autor 6,Afiliação 1,Afiliação 2,Afiliação 3,Afiliação 4,Afiliação 5,Afiliação 6,Url Artigo,Revista
0,2021-07-02,Nível de conhecimento dos profissionais de con...,Objetivo: A preocupação com o meio ambiente é ...,Contador. Contabilidade e Gestão ambiental. Ed...,5,Sirlei Tonello Tisott,Sílvio Paula Ribeiro,Tamires Sousa Araújo,Cinthia da Costa Pontes,Jhonatan Verissímo da Sílva,,Universidade Federal de Mato Grosso do Sul,Universidade Federal de Mato Grosso do Sul,Universidade Federal de Mato Grosso do Sul,Universidade Federal de Mato Grosso do Sul,Universidade Federal de Mato Grosso do Sul,,https://periodicos.ufrn.br/ambiente/article/vi...,Revista Ambiente Contábil
1,2017-01-02,ANÁLISE DO NÍVEL DE CONHECIMENTO DO PROFISSION...,As preocupações provenientes das questões ambi...,Impactos Ambientais. Responsabilidade Social e...,6,Alexandre André Feil,Angela Maria Haberkamp,Adriano José Azeredo,Tatiane Mileidi Orso,Clovis Antônio Kronbauer,Claus Haetinger,Centro Universitário Univates,Centro Universitário UNIVATES e Universidade d...,Centro Universitário UNIVATES,Centro Universitário UNIVATES,Centro Universitário UNIVATES e Universidade d...,,https://periodicos.ufrn.br/ambiente/article/vi...,Revista Ambiente Contábil
2,2013-02-06,ANÁLISE DA PRODUÇÃO CIENTÍFICA BRASILEIRA EM C...,A pesquisa objetivou identificar as mudanças o...,Contabilidade. Gestão. Social. Ambiental. Prod...,2,Pedro Ylunga Costa Da Silva,José Santo Dal Bem Pires,,,,,Universidade Federal do Paraná,Universidade Estadual de Maringá,,,,,https://periodicos.ufrn.br/ambiente/article/vi...,Revista Ambiente Contábil
3,2016-01-03,CONTABILIDADE AMBIENTAL – UMA ANÁLISE DA APLIC...,"A contabilidade ambiental, como uma ramificaçã...",Evidenciação. Contabilidade Ambiental. Educaçã...,2,Walter Nunes Oleiro,Elisabeth Brandão Schmidt,,,,,Universidade Federal do Rio Grande,Universidade Federal do Rio Grande,,,,,https://periodicos.ufrn.br/ambiente/article/vi...,Revista Ambiente Contábil
4,2013-09-10,RELAÇÃO ENTRE DESEMPENHO ECONÔMICO E DESEMPENH...,Com base na Teoria da Sinalização e na Teoria ...,,5,José Flávio Vasconcelos Alves,Márcia Martins Mendes De Luca,Vanessa Ingrid da Costa Cardoso,Alessandra Carvalho de Vasconcelos,Jacqueline Veneroso Alves da Cunha,,Universidade Federal do Ceará,Universidade Federal do Ceará,Universidade Federal do Ceará,Universidade Federal do Ceará,Universidade Federal de Minas Gerais – UFMG,,https://periodicos.ufrn.br/ambiente/article/vi...,Revista Ambiente Contábil


In [24]:
df.to_csv('../data/AMBIENTE.csv', index=False)

In [25]:
df

,Data de Publicação,Título,Resumo,Palavras-chave,Qtd. de Autores,Autor 1,Autor 2,Autor 3,Autor 4,Autor 5,Autor 6,Afiliação 1,Afiliação 2,Afiliação 3,Afiliação 4,Afiliação 5,Afiliação 6,Url Artigo,Revista
0,2021-07-02,Nível de conhecimento dos profissionais de con...,Objetivo: A preocupação com o meio ambiente é ...,Contador. Contabilidade e Gestão ambiental. Ed...,5,Sirlei Tonello Tisott,Sílvio Paula Ribeiro,Tamires Sousa Araújo,Cinthia da Costa Pontes,Jhonatan Verissímo da Sílva,,Universidade Federal de Mato Grosso do Sul,Universidade Federal de Mato Grosso do Sul,Universidade Federal de Mato Grosso do Sul,Universidade Federal de Mato Grosso do Sul,Universidade Federal de Mato Grosso do Sul,,https://periodicos.ufrn.br/ambiente/article/vi...,Revista Ambiente Contábil
1,2017-01-02,ANÁLISE DO NÍVEL DE CONHECIMENTO DO PROFISSION...,As preocupações provenientes das questões ambi...,Impactos Ambientais. Responsabilidade Social e...,6,Alexandre André Feil,Angela Maria Haberkamp,Adriano José Azeredo,Tatiane Mileidi Orso,Clovis Antônio Kronbauer,Claus Haetinger,Centro Universitário Univates,Centro Universitário UNIVATES e Universidade d...,Centro Universitário UNIVATES,Centro Universitário UNIVATES,Centro Universitário UNIVATES e Universidade d...,,https://periodicos.ufrn.br/ambiente/article/vi...,Revista Ambiente Contábil
2,2013-02-06,ANÁLISE DA PRODUÇÃO CIENTÍFICA BRASILEIRA EM C...,A pesquisa objetivou identificar as mudanças o...,Contabilidade. Gestão. Social. Ambiental. Prod...,2,Pedro Ylunga Costa Da Silva,José Santo Dal Bem Pires,,,,,Universidade Federal do Paraná,Universidade Estadual de Maringá,,,,,https://periodicos.ufrn.br/ambiente/article/vi...,Revista Ambiente Contábil
3,2016-01-03,CONTABILIDADE AMBIENTAL – UMA ANÁLISE DA APLIC...,"A contabilidade ambiental, como uma ramificaçã...",Evidenciação. Contabilidade Ambiental. Educaçã...,2,Walter Nunes Oleiro,Elisabeth Brandão Schmidt,,,,,Universidade Federal do Rio Grande,Universidade Federal do Rio Grande,,,,,https://periodicos.ufrn.br/ambiente/article/vi...,Revista Ambiente Contábil
4,2013-09-10,RELAÇÃO ENTRE DESEMPENHO ECONÔMICO E DESEMPENH...,Com base na Teoria da Sinalização e na Teoria ...,,5,José Flávio Vasconcelos Alves,Márcia Martins Mendes De Luca,Vanessa Ingrid da Costa Cardoso,Alessandra Carvalho de Vasconcelos,Jacqueline Veneroso Alves da Cunha,,Universidade Federal do Ceará,Universidade Federal do Ceará,Universidade Federal do Ceará,Universidade Federal do Ceará,Universidade Federal de Minas Gerais – UFMG,,https://periodicos.ufrn.br/ambiente/article/vi...,Revista Ambiente Contábil
5,2017-01-02,RELAÇÃO DE DEPENDÊNCIA ENTRE A DIVULGAÇÃO VOLU...,O objetivo da pesquisa foi analisar a relação ...,Divulgação social e ambiental. ANACOR. Caracte...,2,Maria Audenôra Rufino,Márcia Reis Machado,,,,,UFPB e Unipê,UFPB,,,,,https://periodicos.ufrn.br/ambiente/article/vi...,Revista Ambiente Contábil
6,2021-07-02,Disclosure ambiental dos relatórios corporativ...,Objetivo: Analisar o nível de disclosure ambie...,Relatório de Sustentabilidade. GRI. Disclosure...,3,Claúdia Voges,Cleber Broietti,Suliani Rover,,,,UFSC,UNESPAR,UFSC,,,,https://periodicos.ufrn.br/ambiente/article/vi...,Revista Ambiente Contábil
7,2014-07-25,APLICABILIDADE DO TERMO DE AJUSTAMENTO DE COND...,As organizações se deparam com uma nova realid...,,3,Cristina Keiko Yamaguchi,Kátia Aurora Dalla Libera Soratto,Mariana Gonçalves Mariot,,,,UNIVERSIDADE DO EXTREMO SUL CATARINENSE,,,,,,https://periodicos.ufrn.br/ambiente/article/vi...,Revista Ambiente Contábil
8,2020-01-03,Mapeamento de competências: necessidades de ap...,Objetivo: O estudo tem como objetivo analisar ...,Perícia Contábil. Contabilidade Forense. Compe...,5,Ariel Prates,Jonatas Dutra Sallaberry,Leonardo Flach,Ivam Ricardo Peleias,Romina Batista de Lucena Souza,,Mestre em Contabilidade – Universidade Federal...,Doutorando em Contabilidade pela Universidade ...,Pós,Doutor em Ciências Contábeis pela FEA,Professora Doutora do Programa de Pós,,https://periodic